In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
pitching_23 = pd.read_excel('../../../Downloads/FL 21-23.xlsx', '2023 Frontier League Pitching')
pitching_23['Name'] = pitching_23['Name'].str.replace('*', '')
pitching_23['Year'] = 2023

pitching_22 = pd.read_excel('../../../Downloads/FL 21-23.xlsx', '2022 Frontier League Pitching')
pitching_22['Name'] = pitching_22['Name'].str.replace('*', '')
pitching_22['Year'] = 2022

pitching_21 = pd.read_excel('../../../Downloads/FL 21-23.xlsx', '2021 Frontier League Pitching')
pitching_21['Name'] = pitching_21['Name'].str.replace('*', '')
pitching_21['Year'] = 2021


hitting_23 = pd.read_excel('../../../Downloads/FL 21-23.xlsx', '2023 Frontier League Hitting')
hitting_23['Name'] = hitting_23['Name'].str.replace('*', '')
hitting_23['Year'] = 2023

hitting_22 = pd.read_excel('../../../Downloads/FL 21-23.xlsx', '2022 Frontier League Hitting')
hitting_22['Name'] = hitting_22['Name'].str.replace('*', '')
hitting_22['Year'] = 2022

hitting_21 = pd.read_excel('../../../Downloads/FL 21-23.xlsx', '2021 Frontier League Hitting')
hitting_21['Name'] = hitting_21['Name'].str.replace('*', '')
hitting_21['Year'] = 2021

C:\Users\kellyjc\AppData\Local\Temp\ipykernel_15504\867155827.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pitching_23['Name'] = pitching_23['Name'].str.replace('*', '')
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_15504\867155827.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pitching_22['Name'] = pitching_22['Name'].str.replace('*', '')
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_15504\867155827.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pitching_21['Name'] = pitching_21['Name'].str.replace('*'

In [3]:
def calculate_fip(df):
    f_constant = df.ERA.mean() - (((13 * df.HR.sum()) + (3 * (df.BB.sum() + df.HBP.sum())) - (2 * df.SO.sum())) / df.IP.sum())
    return f_constant + (((13 * df.HR) + (3 * df.BB) - (2 * df.SO)) / df.IP)

pitching_21['FIP'] = calculate_fip(pitching_21)
pitching_22['FIP'] = calculate_fip(pitching_22)
pitching_23['FIP'] = calculate_fip(pitching_23)

In [4]:
hitting_23['1B'] = hitting_23['H'] - (hitting_23['2B'] + hitting_23['3B'] + hitting_23['HR'])
hitting_22['1B'] = hitting_22['H'] - (hitting_22['2B'] + hitting_22['3B'] + hitting_22['HR'])
hitting_21['1B'] = hitting_21['H'] - (hitting_21['2B'] + hitting_21['3B'] + hitting_21['HR'])

In [5]:
woba_vals = {'BB': 0.696, 'HBP': 0.726, '1B': 0.883, '2B': 1.244, '3B': 1.569, 'HR': 2.004, 'SB': 0.200, 'CS': -0.422}
def calculate_woba(df):
    woba = (
        (woba_vals['BB'] * df['BB'] +
        woba_vals['HBP'] * df['HBP'] +
        woba_vals['1B'] * df['1B'] +
        woba_vals['2B'] * df['2B'] +
        woba_vals['3B'] * df['3B'] +
        woba_vals['HR'] * df['HR'] +
        woba_vals['SB'] * df['SB'] +
        woba_vals['CS'] * df['CS']) /
        (df['AB'] + df['BB'] + df['SF'] + df['HBP'])
    )
    return woba

def calculate_babip(df):
    babip = (
        (df['H'] - df['HR']) / (df['AB'] - df['SO'] - df['HR'] + df['SF'])
    )
    return babip

In [6]:
hitting_23['wOBA'] = calculate_woba(hitting_23)
hitting_22['wOBA'] = calculate_woba(hitting_22)
hitting_21['wOBA'] = calculate_woba(hitting_21)

hitting_23['BAbip'] = calculate_babip(hitting_23)
hitting_22['BAbip'] = calculate_babip(hitting_22)
hitting_21['BAbip'] = calculate_babip(hitting_21)

In [7]:
woba_scale = 1.204
hitting_23['wRAA'] = ((hitting_23['wOBA'] - hitting_23['wOBA'].mean()) / woba_scale) * hitting_23.PA
hitting_22['wRAA'] = ((hitting_22['wOBA'] - hitting_22['wOBA'].mean()) / woba_scale) * hitting_22.PA
hitting_21['wRAA'] = ((hitting_21['wOBA'] - hitting_21['wOBA'].mean()) / woba_scale) * hitting_21.PA

In [8]:
hitting_23['WAR'] = hitting_23['wRAA'] / 10
hitting_22['WAR'] = hitting_22['wRAA'] / 10
hitting_21['WAR'] = hitting_21['wRAA'] / 10

In [9]:
def calculate_rates(df):
    df['BB/In'] = df['BB9'] / 9
    df['K/In'] = df['SO9'] / 9
    df['HR/In'] = df['HR9'] / 9
    
    df['BB%'] = df['BB'] / df['BF']
    df['K%'] = df['SO'] / df['BF']
    df['HR%'] = df['HR'] / df['BF']
    
    return df

pitching_21 = calculate_rates(pitching_21)
pitching_22 = calculate_rates(pitching_22)
pitching_23 = calculate_rates(pitching_23)

In [10]:
def calculate_rates_hitters(df):
    df['BB/PA'] = df['BB'] / df['PA']
    df['K/PA'] = df['SO'] / df['PA']
    df['HR/PA'] = df['HR'] / df['PA']
    
    df['1B/PA'] = df['1B'] / df['PA']
    df['2B/PA'] = df['2B'] / df['PA']
    df['3B/PA'] = df['3B'] / df['PA']
    
    return df

In [11]:
hitting_21 = calculate_rates_hitters(hitting_21)
hitting_22 = calculate_rates_hitters(hitting_22)
hitting_23 = calculate_rates_hitters(hitting_23)

In [12]:
fl = pd.read_csv('../../../Desktop/Joliet Slammers/AutomatedPitchTagging/AutotaggedCSVs/2023 Frontier League Autotagged.csv')

In [13]:
fl.Pitcher = fl.Pitcher.replace({'Luis Perez': 'Luis Pérez', 'Damon Casetta-stubbs': 'Damon Casetta-Stubbs', 'Damon Casetta-stubs': 'Damon Casetta-Stubbs'})

In [14]:
fl.PlayResult.unique()

array([nan, 'Single', 'Out', 'Sacrifice', 'StrikeoutSwinging', 'HomeRun',
       'Error', 'Walk', 'StrikeoutLooking', 'Double', 'FieldersChoice',
       'Triple'], dtype=object)

In [15]:
for p in pitching_23.Name.unique():
    p_data = fl[fl['Pitcher'] == p]
    bf = len(p_data[['Date', 'Batter', 'Inning', 'Outs']].drop_duplicates())
    k_percent = (p_data['PlayResult'].eq('StrikeoutLooking').sum() +
                 p_data['PlayResult'].eq('StrikeoutSwinging').sum()) / bf
    bb_percent = p_data['PlayResult'].eq('Walk').sum() / bf
    hr_percent = p_data['PlayResult'].eq('HomeRun').sum() / bf

    pitching_23.loc[pitching_23['Name'] == p, 'K%'] = k_percent
    pitching_23.loc[pitching_23['Name'] == p, 'BB%'] = bb_percent
    pitching_23.loc[pitching_23['Name'] == p, 'HR%'] = hr_percent

C:\Users\kellyjc\AppData\Local\Temp\ipykernel_15504\2929819247.py:4: RuntimeWarning: invalid value encountered in longlong_scalars
  k_percent = (p_data['PlayResult'].eq('StrikeoutLooking').sum() +
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_15504\2929819247.py:6: RuntimeWarning: invalid value encountered in longlong_scalars
  bb_percent = p_data['PlayResult'].eq('Walk').sum() / bf
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_15504\2929819247.py:7: RuntimeWarning: invalid value encountered in longlong_scalars
  hr_percent = p_data['PlayResult'].eq('HomeRun').sum() / bf
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_15504\2929819247.py:4: RuntimeWarning: invalid value encountered in longlong_scalars
  k_percent = (p_data['PlayResult'].eq('StrikeoutLooking').sum() +
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_15504\2929819247.py:6: RuntimeWarning: invalid value encountered in longlong_scalars
  bb_percent = p_data['PlayResult'].eq('Walk').sum() / bf
C:\Users\kellyjc\AppData\Local\Tem

In [16]:
pitching = pd.concat([pitching_23, pitching_22, pitching_21])
hitting = pd.concat([hitting_23, hitting_22, hitting_21])

In [17]:
hitting['SB%'] = hitting['SB'] / hitting['PA']

In [18]:
pitching_weights = {2023: 3, 2022: 2, 2021: 1}
hitting_weights = {2023: 5, 2022: 4, 2021: 3}

In [19]:
def calculate_pitching_metrics(group):
    divisor = 0
    for year, group_data in group.groupby('Year'):
        divisor += pitching_weights[year]
        
    result = {
        'K/In': (group['K/In'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'BB/In': (group['BB/In'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'HR/In': (group['HR/In'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'ERA': (group['ERA'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'FIP': (group['FIP'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'WHIP': (group['WHIP'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'K%': (group['K%'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'BB%': (group['BB%'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'HR%': (group['HR%'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum()
    }

    return pd.Series(result, index=['K/In', 'BB/In', 'HR/In', 'ERA', 'FIP', 'WHIP', 'K%', 'BB%', 'HR%'])


pitcher_proj = pitching.groupby('Name').apply(calculate_pitching_metrics).reset_index()
pitcher_proj['IP'] = pitching.groupby('Name')['IP'].sum().values

In [20]:
def calculate_hitting_metrics(group):
    divisor = 0
    for year, group_data in group.groupby('Year'):
        divisor += hitting_weights[year]
        
    result = {
        'K/PA': (group['K/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'BB/PA': (group['BB/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'HR/PA': (group['HR/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'wOBA': (group['wOBA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'wRAA': (group['wRAA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'BAbip': (group['BAbip'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'OBP': (group['OBP'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'SLG': (group['SLG'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'SB%': (group['SB%'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'BA': (group['BA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        '1B/PA': (group['1B/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(), 
        '2B/PA': (group['2B/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        '3B/PA': (group['3B/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum()
    }

    return pd.Series(result, index=['K/PA', 'BB/PA', 'HR/PA', 'wOBA', 'BAbip', 'wRAA', 'BA', 'SLG', 'OBP', 'SB%', '1B/PA', '2B/PA', '3B/PA'])

hitter_proj = hitting.groupby('Name').apply(calculate_hitting_metrics).reset_index()
hitter_proj['PA'] = hitting.groupby('Name')['PA'].sum().values

In [21]:
hitter_proj

,Name,K/PA,BB/PA,HR/PA,wOBA,BAbip,wRAA,BA,SLG,OBP,SB%,1B/PA,2B/PA,3B/PA,PA
0,A.J. Wright,0.222140,0.130286,0.031714,0.378884,0.345803,1.571240,0.281444,0.469111,0.396444,0.016713,0.144001,0.053428,0.003571,821
1,Aaron Altherr,0.268698,0.110803,0.058172,0.403335,0.358974,9.194284,0.294000,0.555000,0.385000,0.022161,0.146814,0.044321,0.002770,361
2,Abdiel Diaz,0.148388,0.125888,0.010263,0.332698,0.314532,-8.836353,0.270000,0.352875,0.370000,0.065242,0.181770,0.034919,0.002143,622
3,Alberti Chavez,0.137845,0.032581,0.010025,0.322010,0.353846,-18.168587,0.313000,0.411000,0.336000,0.025063,0.230576,0.052632,0.005013,399
4,Alec Craig,0.162222,0.204444,0.017778,0.380864,0.317164,1.506196,0.267000,0.394000,0.431000,0.066667,0.151111,0.031111,0.006667,450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Zach Biermann,0.224277,0.085947,0.034555,0.373148,0.356150,1.781009,0.293714,0.489286,0.376143,0.012011,0.165016,0.048439,0.009057,523
219,Zach Huffins,0.240000,0.045714,0.011429,0.285423,0.344262,-12.681356,0.268000,0.335000,0.303000,0.045714,0.211429,0.028571,0.000000,175
220,Zach Kirtley,0.229665,0.145933,0.038278,0.356175,0.302905,0.168868,0.256000,0.451000,0.368000,0.002392,0.126794,0.047847,0.000000,418
221,Zach Racusin,0.123322,0.089003,0.011055,0.349706,0.348728,-4.226253,0.310917,0.404417,0.380500,0.047918,0.218545,0.038763,0.005130,792


In [22]:
hitter_proj.PA.mean()

411.21973094170403

In [23]:
avg_Kp = pitcher_proj['K/In'].mean()
avg_BBp = pitcher_proj['BB/In'].mean()
avg_HRp = pitcher_proj['HR/In'].mean()
avg_ERA = pitcher_proj['ERA'].mean()
avg_FIP = pitcher_proj['FIP'].mean()
avg_WHIP = pitcher_proj['WHIP'].mean()
avg_Kpp = pitcher_proj['K%'].mean()
avg_BBpp = pitcher_proj['BB%'].mean()
avg_HRpp = pitcher_proj['HR%'].mean()


mask = pitcher_proj['IP'] < 100

pitcher_proj.loc[mask, 'K/In'] = (pitcher_proj.loc[mask, 'K/In'] * pitcher_proj.loc[mask, 'IP'] +
                                (100 - pitcher_proj.loc[mask, 'IP']) * avg_Kp) / 100

pitcher_proj.loc[mask, 'BB/In'] = (pitcher_proj.loc[mask, 'BB/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_BBp) / 100

pitcher_proj.loc[mask, 'HR/In'] = (pitcher_proj.loc[mask, 'HR/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_HRp) / 100

pitcher_proj.loc[mask, 'ERA'] = (pitcher_proj.loc[mask, 'ERA'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_ERA) / 100

pitcher_proj.loc[mask, 'FIP'] = (pitcher_proj.loc[mask, 'FIP'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_FIP) / 100

pitcher_proj.loc[mask, 'WHIP'] = (pitcher_proj.loc[mask, 'WHIP'] * pitcher_proj.loc[mask, 'IP'] +
                                  (100 - pitcher_proj.loc[mask, 'IP']) * avg_WHIP) / 100

pitcher_proj.loc[mask, 'K%'] = (pitcher_proj.loc[mask, 'K%'] * pitcher_proj.loc[mask, 'IP'] +
                                (100 - pitcher_proj.loc[mask, 'IP']) * avg_Kpp) / 100

pitcher_proj.loc[mask, 'BB%'] = (pitcher_proj.loc[mask, 'BB%'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_BBpp) / 100

pitcher_proj.loc[mask, 'HR%'] = (pitcher_proj.loc[mask, 'HR%'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_HRpp) / 100


pitcher_proj['Reliability'] = 1 - (100 - pitcher_proj['IP']).clip(0) / 100

In [24]:
avg_Kp = hitter_proj['K/PA'].mean()
avg_BBp = hitter_proj['BB/PA'].mean()
avg_HRp = hitter_proj['HR/PA'].mean()
avg_s = hitter_proj['1B/PA'].mean()
avg_d = hitter_proj['2B/PA'].mean()
avg_t = hitter_proj['3B/PA'].mean()
avg_woba = hitter_proj['wOBA'].mean()
avg_wraa = hitter_proj['wRAA'].mean()
avg_babip = hitter_proj['BAbip'].mean()
avg_ba = hitter_proj['BA'].mean()
avg_slg = hitter_proj['SLG'].mean()
avg_obp = hitter_proj['OBP'].mean()
avg_sb = hitter_proj['SB%'].mean()


mask = hitter_proj['PA'] < 400

hitter_proj.loc[mask, 'K/PA'] = (hitter_proj.loc[mask, 'K/PA'] * hitter_proj.loc[mask, 'PA'] +
                                (400 - hitter_proj.loc[mask, 'PA']) * avg_Kp) / 400

hitter_proj.loc[mask, 'BB/PA'] = (hitter_proj.loc[mask, 'BB/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_BBp) / 400

hitter_proj.loc[mask, 'HR/PA'] = (hitter_proj.loc[mask, 'HR/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_HRp) / 400

hitter_proj.loc[mask, 'wOBA'] = (hitter_proj.loc[mask, 'wOBA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_woba) / 400

hitter_proj.loc[mask, 'wRAA'] = (hitter_proj.loc[mask, 'wRAA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_wraa) / 400

hitter_proj.loc[mask, 'BAbip'] = (hitter_proj.loc[mask, 'BAbip'] * hitter_proj.loc[mask, 'PA'] +
                                  (400 - hitter_proj.loc[mask, 'PA']) * avg_babip) / 400

hitter_proj.loc[mask, 'BA'] = (hitter_proj.loc[mask, 'BA'] * hitter_proj.loc[mask, 'PA'] +
                                (400 - hitter_proj.loc[mask, 'PA']) * avg_ba) / 400

hitter_proj.loc[mask, 'SLG'] = (hitter_proj.loc[mask, 'SLG'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_slg) / 400

hitter_proj.loc[mask, 'OBP'] = (hitter_proj.loc[mask, 'OBP'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_obp) / 400

hitter_proj.loc[mask, '1B/PA'] = (hitter_proj.loc[mask, '1B/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_s) / 400

hitter_proj.loc[mask, '2B/PA'] = (hitter_proj.loc[mask, '2B/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_d) / 400

hitter_proj.loc[mask, '3B/PA'] = (hitter_proj.loc[mask, '3B/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_t) / 400

hitter_proj['Reliability'] = 1 - (400 - hitter_proj['PA']).clip(0) / 400

In [25]:
hitter_proj

,Name,K/PA,BB/PA,HR/PA,wOBA,BAbip,wRAA,BA,SLG,OBP,SB%,1B/PA,2B/PA,3B/PA,PA,Reliability
0,A.J. Wright,0.222140,0.130286,0.031714,0.378884,0.345803,1.571240,0.281444,0.469111,0.396444,0.016713,0.144001,0.053428,0.003571,821,1.0000
1,Aaron Altherr,0.260465,0.109477,0.055316,0.399804,0.357572,8.324258,0.294239,0.546143,0.384142,0.022161,0.149562,0.044725,0.002989,361,0.9025
2,Abdiel Diaz,0.148388,0.125888,0.010263,0.332698,0.314532,-8.836353,0.270000,0.352875,0.370000,0.065242,0.181770,0.034919,0.002143,622,1.0000
3,Alberti Chavez,0.137961,0.032743,0.010072,0.322123,0.353823,-18.122488,0.312959,0.411133,0.336101,0.025063,0.230437,0.052621,0.005013,399,0.9975
4,Alec Craig,0.162222,0.204444,0.017778,0.380864,0.317164,1.506196,0.267000,0.394000,0.431000,0.066667,0.151111,0.031111,0.006667,450,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Zach Biermann,0.224277,0.085947,0.034555,0.373148,0.356150,1.781009,0.293714,0.489286,0.376143,0.012011,0.165016,0.048439,0.009057,523,1.0000
219,Zach Huffins,0.208642,0.074676,0.021249,0.331378,0.344448,-5.395691,0.284003,0.407649,0.344176,0.045714,0.190937,0.039760,0.002820,175,0.4375
220,Zach Kirtley,0.229665,0.145933,0.038278,0.356175,0.302905,0.168868,0.256000,0.451000,0.368000,0.002392,0.126794,0.047847,0.000000,418,1.0000
221,Zach Racusin,0.123322,0.089003,0.011055,0.349706,0.348728,-4.226253,0.310917,0.404417,0.380500,0.047918,0.218545,0.038763,0.005130,792,1.0000


In [26]:
projections_24 = pitching_23[['Name', 'Age', 'Tm']].merge(pitcher_proj, on='Name', how='left')
projections_hit_24 = hitting_23[['Name', 'Age', 'Tm']].merge(hitter_proj, on='Name', how='left')

In [27]:
for p in projections_24.Name.unique():
    p_data = pitching_22[pitching_22['Name'] == p]
    ip_23 = pitching_23[pitching_23['Name'] == p].IP.values[0]
    if len(p_data) == 0:
        proj_ip = .5 * ip_23 + 30
    else: 
        proj_ip = .5 * ip_23 + .1 * p_data['IP'].values[0] + 30
    projections_24.loc[projections_24['Name'] == p, 'IP'] = proj_ip

In [28]:
for h in projections_hit_24['Name'].unique():
    h_data = hitting_22[hitting_22['Name'] == h]
    pa_23 = hitting_23[hitting_23['Name'] == h]['PA'].values[0]
    if len(h_data) == 0:
        proj_pa = 0.5 * pa_23 + 100
    else:
        proj_pa = 0.5 * pa_23 + 0.1 * h_data['PA'].values[0] + 100
    projections_hit_24.loc[projections_hit_24['Name'] == h, 'PA'] = proj_pa

In [29]:
projections_24

,Name,Age,Tm,K/In,BB/In,HR/In,ERA,FIP,WHIP,K%,BB%,HR%,IP,Reliability
0,Cole Cook,25,JOL,0.946667,0.213333,0.033333,2.626000,2.495508,1.053600,0.224405,0.038934,0.005740,110.21,1.000
1,Joey Gonzalez,26,GAT,1.006667,0.413333,0.153333,4.518000,4.557265,1.420400,0.228078,0.063559,0.031686,93.62,1.000
2,Turner Larkins,27,JOL,0.711111,0.277778,0.066667,4.120000,3.718147,1.323000,0.151436,0.023499,0.015666,87.60,1.000
3,Collin Sullivan,25,GAT,1.222222,0.322222,0.077778,2.690000,2.851605,1.106000,0.301518,0.036876,0.021692,85.05,1.000
4,Steven Fuentes,26,QUE,0.647801,0.358528,0.059368,2.498817,3.953702,1.219126,0.150510,0.071571,0.014494,76.00,0.920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Cam Aufderheide,26,JOL,0.738889,0.433333,0.083333,4.386667,4.224703,1.490500,0.162033,0.088295,0.015765,89.07,1.000
96,Jon Beymer,24,EVN,1.065843,0.487159,0.056239,4.018690,3.407934,1.450461,0.211933,0.069428,0.012204,59.00,0.580
97,Garrison Bryant,24,TRC,0.773208,0.373672,0.165843,5.525858,5.157231,1.502161,0.160766,0.057281,0.032064,66.55,0.731
98,Stephen Chamblee,27,LEC,0.769683,0.302966,0.180037,5.954462,5.098644,1.374541,0.166089,0.042650,0.037243,65.60,0.712


In [30]:
projections_24['Age'] = pd.to_numeric(projections_24['Age'], errors='coerce')
projections_24['Age'] += 1
projections_24.loc[projections_24['Age'] >= 29, 'AgeAdj'] = 1 / (1 + (((projections_24['Age'] - 29) - 29) * 0.003))
projections_24.loc[projections_24['Age'] < 29, 'AgeAdj'] = 1 + ((29 - (projections_24['Age'] - 29)) * 0.006)

In [31]:
projections_hit_24['Age'] = pd.to_numeric(projections_hit_24['Age'], errors='coerce')
projections_hit_24['Age'] += 1
projections_hit_24.loc[projections_hit_24['Age'] >= 29, 'AgeAdj'] = 1 / (1 + (((projections_hit_24['Age'] - 29) - 29) * 0.003))
projections_hit_24.loc[projections_hit_24['Age'] < 29, 'AgeAdj'] = 1 + ((29 - (projections_hit_24['Age'] - 29)) * 0.006)

In [32]:
projections_24['IP'] *= projections_24['AgeAdj']
projections_hit_24['PA'] *= projections_hit_24['AgeAdj']

In [33]:
projections_24 = projections_24.dropna(subset=['IP'])
projections_hit_24 = projections_hit_24.dropna(subset=['PA'])

In [34]:
projections_24['K'] = projections_24['K/In'] * projections_24['IP']
projections_24['BB'] = projections_24['BB/In'] * projections_24['IP']
projections_24['HR'] = projections_24['HR/In'] * projections_24['IP']

projections_24['K'] = projections_24['K'].apply(math.floor)
projections_24['BB'] = projections_24['BB'].apply(math.floor)
projections_24['HR'] = projections_24['HR'].apply(math.floor)

C:\Users\kellyjc\AppData\Local\Temp\ipykernel_15504\1334175161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projections_24['K'] = projections_24['K/In'] * projections_24['IP']
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_15504\1334175161.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projections_24['BB'] = projections_24['BB/In'] * projections_24['IP']
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_15504\1334175161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [36]:
projections_hit_24['SB'] = projections_hit_24['SB%'] * projections_hit_24['PA']
projections_hit_24['HR'] = projections_hit_24['HR/PA'] * projections_hit_24['PA']
projections_hit_24['K'] = projections_hit_24['K/PA'] * projections_hit_24['PA']
projections_hit_24['BB'] = projections_hit_24['BB/PA'] * projections_hit_24['PA']
projections_hit_24['1B'] = projections_hit_24['1B/PA'] * projections_hit_24['PA']
projections_hit_24['2B'] = projections_hit_24['2B/PA'] * projections_hit_24['PA']
projections_hit_24['3B'] = projections_hit_24['3B/PA'] * projections_hit_24['PA']
projections_hit_24['SO'] = projections_hit_24['K/PA'] * projections_hit_24['PA']

projections_hit_24['K'] = projections_hit_24['K'].apply(math.floor)
projections_hit_24['BB'] = projections_hit_24['BB'].apply(math.floor)
projections_hit_24['HR'] = projections_hit_24['HR'].apply(math.floor)
projections_hit_24['SB'] = projections_hit_24['SB'].apply(math.floor)
projections_hit_24['1B'] = projections_hit_24['1B'].apply(math.floor)
projections_hit_24['2B'] = projections_hit_24['2B'].apply(math.floor)
projections_hit_24['3B'] = projections_hit_24['3B'].apply(math.floor)
projections_hit_24['SO'] = projections_hit_24['SO'].apply(math.floor)

In [37]:
projections_24 = projections_24[['Name', 'Age', 'Tm', 'ERA', 'FIP', 'IP', 'K', 'BB', 'HR', 'K%', 'BB%', 'HR%', 'Reliability']]

In [38]:
projections_hit_24 = projections_hit_24[['Name', 'Age', 'Tm', 'wOBA', 'wRAA', 'BA', 'SLG', 'OBP', 'BAbip', 'SB', 'HR', 'BB', '1B', '2B', '3B', 'SO', 'Reliability']]

In [39]:
projections_hit_24['bat_WAR'] = projections_hit_24['wRAA'] / 10

C:\Users\kellyjc\AppData\Local\Temp\ipykernel_15504\4075359035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projections_hit_24['bat_WAR'] = projections_hit_24['wRAA'] / 10


In [40]:
projections_hit_24.sort_values('bat_WAR', ascending=False).head(20)

,Name,Age,Tm,wOBA,wRAA,BA,SLG,OBP,BAbip,SB,HR,BB,1B,2B,3B,SO,Reliability,bat_WAR
2,James Nelson,26,NJJ,0.503192,46.831650,0.388000,0.714000,0.482000,0.466387,29,26,46,74,20,1,84,1.0000,4.683165
5,Jairus Richards,27,GAT,0.457443,28.937935,0.340000,0.525000,0.472000,0.391667,66,14,59,73,8,0,61,1.0000,2.893794
13,Noah Myers,24,EVN,0.444530,25.962529,0.322000,0.568000,0.438000,0.382470,36,15,56,59,20,4,77,1.0000,2.596253
3,Peter Zimmermann,26,GAT,0.464728,24.962415,0.348413,0.633080,0.460564,0.387417,11,18,44,52,16,0,56,0.8050,2.496242
19,Keon Barnum,31,NJJ,0.439219,20.548579,0.314485,0.621759,0.423986,0.338937,0,23,49,49,6,0,67,0.9225,2.054858
35,Brennan Price,28,FLR,0.435073,19.314412,0.306222,0.595333,0.435222,0.345639,2,23,59,52,19,0,82,1.0000,1.931441
52,Jakob Goldfarb,28,TRC,0.423393,16.481740,0.293086,0.532254,0.421830,0.319423,23,15,40,46,18,0,61,0.9750,1.648174
21,Josh Rehwaldt,27,NJJ,0.418353,15.791946,0.314583,0.610250,0.384000,0.370666,8,28,41,64,23,0,108,1.0000,1.579195
33,Patrick Kivlehan,34,RKL,0.415538,14.348632,0.304000,0.586000,0.392000,0.315385,7,18,36,45,19,1,54,1.0000,1.434863
1,Carson McCusker,26,TRC,0.460215,12.772989,0.366293,0.661324,0.430896,0.395702,2,14,22,44,12,0,38,0.7300,1.277299


In [41]:
projections_24.to_csv('2024 Pitcher Projections.csv')

In [42]:
projections_hit_24.to_csv('2024 Hitter Projections.csv')